In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext autoreload
%autoreload 2

from caltech256 import Caltech256
from deep_cnn_structures import train_model, test_model, each_class_accuracy

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'visual': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}
'''
# Replace Resize() with below in pytorch 0.2
transforms.Scale(256),
'''

data_dir = 'data/256_ObjectCategories'
#data_dir = '/datasets/Caltech256/256_ObjectCategories'

In [3]:
caltech256_train = Caltech256(data_dir, data_transforms['train'], train=True)
caltech256_test = Caltech256(data_dir, data_transforms['test'], train=False)

In [ ]:
vgg16 = models.vgg16_bn(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=256)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier._modules['6'].parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30)

vgg16 = nn.DataParallel(vgg16)
vgg16 = vgg16.cuda()
#vgg16.load_state_dict(torch.load('vgg16_tf_2_epochs'))

In [ ]:
vgg16 = train_model(vgg16, caltech256_train, criterion, optimizer, num_epochs=5, batch_size=16, scheduler)

In [ ]:
test_model(vgg16, caltech256_test)

In [ ]:
each_class_accuracy(vgg16, caltech256_test)

In [12]:
torch.save(vgg16.state_dict(), 'saved_models/vgg16_tf_5_epochs')

In [13]:
'''testiter = iter(testloader)
images, labels = testiter.next()
outputs = vgg16(Variable(images.cuda()))
_, pred = torch.max(outputs.data, 1)
print(pred, labels)'''

'testiter = iter(testloader)\nimages, labels = testiter.next()\noutputs = vgg16(Variable(images.cuda()))\n_, pred = torch.max(outputs.data, 1)\nprint(pred, labels)'